In [2]:
import os
import optuna
import mlflow
import mlflow.sklearn
from mlflow.client import MlflowClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import optuna
from catboost import CatBoostRegressor
from optuna.integration.mlflow import MLflowCallback
import numpy as np
import random
from optuna.samplers import TPESampler
from numpy import median, array 

/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "catb00st_optuna" # напишите название вашего эксперимента
RUN_NAME = "optuna"

FS_ASSETS = 'assets/optuna'
os.makedirs(FS_ASSETS, exist_ok=True)


pd.options.display.max_columns = 100
pd.options.display.max_rows = 64

In [4]:
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net" #endpoint бакета от YandexCloud
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID") # получаем id ключа бакета, к которому подключён MLFlow, из .env
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY") # получаем ключ бакета, к которому подключён MLFlow, из .env

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

In [5]:
def get_artifact_runID(name):
    exp_ids=[]
    exps = client.search_experiments()
    got = False
    for e in exps:
        if not got:
            exp_ids.append(e.experiment_id)
            for e_id in exp_ids:
                runs = client.search_runs(e_id)
                for run in runs:
                    if run.info.status !='FAILED' and not got:
                        artifacts = client.list_artifacts(run.info.run_id)
                        #print(artifacts)
                        try:
                            
                            if name in str(artifacts):
                                run_id = run.info.run_id
                                print(run_id)
                                got = True
                        except:
                            pass
    return run_id

In [6]:
client = MlflowClient()
# Получаем список артефактов
the_run_id = get_artifact_runID("f_test.csv")

local_path = client.download_artifacts(the_run_id, "f_test.csv", 'data')
test = pd.read_csv(local_path)
test.sample(5)

2672a8996f4249ea82958891c914e4e3


,cats_transformer__encoder__nearest_metro,numeric__kitchen_area living_area^2,numeric__distance_to_metro,numeric__living_area^3,numeric__living_area,numeric__distance_to_metro^2,cats_transformer__bins_to_cats__build_year,numeric__kitchen_area living_area,cats_transformer__bins_to_cats__floors_total,cats_transformer__encoder__rooms,y
29144,1.623018e+07,18464.767774,6.827563,89915.393148,44.800000,46.615614,1.134803e+07,412.159993,1.208004e+07,1.246098e+07,11500000
6416,1.265097e+07,5832.000000,18.103179,19683.000000,27.000000,327.725078,1.368338e+07,216.000000,1.491554e+07,9.123777e+06,8000000
18975,1.325821e+07,9600.000000,8.052151,64000.000000,40.000000,64.837130,1.028430e+07,240.000000,1.159648e+07,1.246098e+07,9400000
26202,1.325821e+07,25704.409277,3.997304,126506.020978,50.200002,15.978437,1.444640e+07,512.040008,1.159648e+07,1.246098e+07,16800000
30201,1.517227e+07,4375.000000,4.230069,15625.000000,25.000000,17.893482,1.444640e+07,175.000000,1.159648e+07,9.123777e+06,9500000


In [7]:
the_run_id = get_artifact_runID("f_train.csv")

local_path = client.download_artifacts(the_run_id, "f_train.csv", 'data')
train = pd.read_csv(local_path)
train.sample(5)

2672a8996f4249ea82958891c914e4e3


,cats_transformer__encoder__nearest_metro,numeric__kitchen_area living_area^2,numeric__distance_to_metro,numeric__living_area^3,numeric__living_area,numeric__distance_to_metro^2,cats_transformer__bins_to_cats__build_year,numeric__kitchen_area living_area,cats_transformer__bins_to_cats__floors_total,cats_transformer__encoder__rooms,y
10448,1.321513e+07,3742.847963,6.817552,11852.352297,22.800000,46.479020,1.028533e+07,164.159997,1.159679e+07,9.123555e+06,6200000
26452,1.542748e+07,8715.924611,2.688810,54010.158950,37.800002,7.229699,1.443717e+07,230.580006,1.159679e+07,1.246363e+07,10700000
78290,1.319106e+07,37558.438226,1.680423,269586.116897,64.599998,2.823823,1.138655e+07,581.399986,1.208852e+07,1.566804e+07,10000000
14258,1.065513e+07,20469.530900,19.420567,41781.925756,34.700001,377.158432,1.229063e+07,589.900013,1.208643e+07,1.246363e+07,7400000
17852,1.240468e+07,3810.240231,13.217156,16003.009453,25.200001,174.693206,1.443717e+07,151.200005,1.159679e+07,9.123576e+06,6300000


In [8]:
X_train = train.drop(['y'], axis=1)
y_train = train['y']
X_test = test.drop(['y'], axis=1)
y_test = test['y']

In [9]:
def objective(trial, X_train, X_test, y_train, y_test, experiment_id):
    """Objective function to be optimized by Optuna"""
    # Use nested runs to track each trial in MLflow
    with mlflow.start_run(experiment_id = experiment_id, nested=True):
        # Define hyperparameters to optimize
        params = {
            'iterations': trial.suggest_int('iterations', 100, 600),
            'depth': trial.suggest_int('depth', 2, 16),
            'min_child_samples': trial.suggest_int('min_child_samples', 2, 10),
            'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.05, 1.0),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            'loss_function': 'RMSE',
            'random_state': 42,
            'silent': True
        }
        
        # Log parameters
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)
        
        # Create and train the model
        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train)
        
        # Make predictions and evaluate
        y_pred = model.predict(X_test)
        metrics ={
            'neg_mean_absolute_error': - mean_absolute_error(y_pred, y_test),
            'neg_root_mean_squared_error': - mean_squared_error(y_pred, y_test),
            'r2': r2_score(y_pred, y_test),
            'neg_mean_absolute_percentage_error': - mean_absolute_percentage_error(y_pred, y_test)
        }
        
        # Log metrics
        mlflow.log_metrics(metrics)
        
        # Log model
        mlflow.catboost.log_model(model, 'model')
        
        # Log feature importance
        importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        fig, ax = plt.figure(figsize=(10, 6)), plt.axes()
        importance.plot.bar(x='feature', y='importance', ax=ax)
        plt.title('Feature Importance')
        plt.tight_layout()
        plt.savefig('feature_importance.png')
        mlflow.log_artifact('feature_importance.png')
        plt.close()
        
        print(f"Trial {trial.number}: RMSE={mean_squared_error(y_pred, y_test):.4f}, MAE={mean_absolute_error(y_pred, y_test):.4f}, R2={r2_score(y_pred, y_test):.4f}, MAPE%={mean_absolute_percentage_error(y_pred, y_test):.4f}")
        
        return mean_squared_error(y_pred, y_test)  # We want to minimize RMSE


In [ ]:
def run_optimization(experiment_id):
    """Run the complete optimization process"""
    # Setup MLflow with S3
    
    # Create and run Optuna study with MLflow tracking
    with mlflow.start_run(run_name="optuna_optimization", experiment_id=experiment_id):
        mlflow.log_param("optimizer", "optuna")
        mlflow.log_param("n_trials", 10)
        
        # Create study
        study = optuna.create_study(direction='minimize', 
                                    study_name='cb_hyperparameter_optimization',
                                    pruner=optuna.pruners.MedianPruner())
        
        # Optimize with partial function to pass dataset
        study.optimize(lambda trial: objective(trial, X_train, X_test, y_train, y_test, experiment_id), 
                       n_trials=10)
        
        # Log best trial information
        mlflow.log_params({f"best_{k}": v for k, v in study.best_params.items()})
        #mlflow.log_metric("best_rmse", study.best_value)
        
        # Print results
        print("\n" + "="*50)
        print("Best trial:")
        print(f"  Value (RMSE): {study.best_value:.4f}")
        print("  Params:")
        for key, value in study.best_params.items():
            print(f"    {key}: {value}")
        
        # Train the best model
        best_model = CatBoostRegressor(**study.best_params)
        best_model.fit(X_train, y_train)
        
        # Log the best model
        mlflow.sklearn.log_model(best_model, "optuna")
        
        # Generate and log optimization visualizations
        try:
            fig = optuna.visualization.plot_optimization_history(study)
            fig.write_image("optimization_history.png")
            mlflow.log_artifact("optimization_history.png")
            
            fig = optuna.visualization.plot_param_importances(study)
            fig.write_image("param_importances.png")
            mlflow.log_artifact("param_importances.png")
            
            fig = optuna.visualization.plot_slice(study)
            fig.write_image("slice_plot.png")
            mlflow.log_artifact("slice_plot.png")
        except Exception as e:
            print(f"Visualization error: {e}")
        
        print("\nOptimization completed. All results are logged to MLflow.")
        print(f"Best model saved with parameters: {study.best_params}")

if __name__ == "__main__":
    experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if not experiment:
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
    else:
        experiment_id = experiment.experiment_id
    print(experiment)
    run_optimization(experiment_id)

<Experiment: artifact_location='s3://s3-student-mle-20250130-833968fcc1/20', creation_time=1741694413161, experiment_id='20', last_update_time=1741694413161, lifecycle_stage='active', name='catb00st_optuna', tags={}>


[I 2025-03-11 13:38:19,935] A new study created in memory with name: cb_hyperparameter_optimization
[I 2025-03-11 13:38:32,791] Trial 0 finished with value: 6894244488828.594 and parameters: {'iterations': 206, 'depth': 8, 'min_child_samples': 4, 'colsample_bylevel': 0.9120607611667203, 'subsample': 0.6865255566465035}. Best is trial 0 with value: 6894244488828.594.


Trial 0: RMSE=6894244488828.5938, MAE=2009646.8813, R2=0.6213, MAPE%=0.1656


/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[I 2025-03-11 13:38:40,672] Trial 1 finished with value: 7379397219179.799 and parameters: {'iterations': 597, 'depth

Trial 1: RMSE=7379397219179.7988, MAE=2071857.6277, R2=0.5659, MAPE%=0.1687


...

Trial 8: RMSE=7666731800866.4775, MAE=2109699.6895, R2=0.5439, MAPE%=0.1710


/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[I 2025-03-11 13:52:59,995] Trial 9 finished with value: 6473304393247.93 and parameters: {'iterations': 578, 'depth'

Trial 9: RMSE=6473304393247.9297, MAE=1943018.2886, R2=0.6465, MAPE%=0.1597

Best trial:
  Value (RMSE): 6473304393247.9297
  Params:
    iterations: 578
    depth: 14
    min_child_samples: 6
    colsample_bylevel: 0.48937975070584505
    subsample: 0.4259784517693547
Learning rate set to 0.13132
0:	learn: 4601267.1469270	total: 198ms	remaining: 1m 54s
1:	learn: 4269011.9123614	total: 520ms	remaining: 2m 29s
2:	learn: 3987845.0947216	total: 724ms	remaining: 2m 18s
3:	learn: 3757621.4642224	total: 956ms	remaining: 2m 17s
4:	learn: 3569658.4129250	total: 1.17s	remaining: 2m 14s
5:	learn: 3415124.0911319	total: 1.45s	remaining: 2m 18s
6:	learn: 3290890.4329448	total: 1.74s	remaining: 2m 21s
7:	learn: 3190722.3427914	total: 1.93s	remaining: 2m 17s
8:	learn: 3103473.3817407	total: 2.13s	remaining: 2m 14s
9:	learn: 3035006.5439925	total: 2.42s	remaining: 2m 17s
10:	learn: 2979587.7711301	total: 2.65s	remaining: 2m 16s
11:	learn: 2929369.0874972	total: 2.9s	remaining: 2m 16s
12:	learn: 28925

/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/mle-user/mle_projects/.venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(



Optimization completed. All results are logged to MLflow.
Best model saved with parameters: {'iterations': 578, 'depth': 14, 'min_child_samples': 6, 'colsample_bylevel': 0.48937975070584505, 'subsample': 0.4259784517693547}
